# Explore Delta Lake

In this notebook, you'll explore how to use Delta Lake in a Databricks Spark cluster.

## Ingest data

Use the **&#9656; Run Cell** menu option at the top-right of the following cell to run it and download a data file into the Databricks file system (DBFS).

In [ ]:
%sh
rm -r /dbfs/data
rm -r /dbfs/delta
mkdir /dbfs/data
wget -O /dbfs/data/products.csv https://raw.githubusercontent.com/MicrosoftLearning/dp-203-azure-data-engineer/master/Allfiles/labs/25/data/products.csv

Now that you've ingested the data, you can load it into a Spark dataframe from the Databricks file system (DBFS)

In [ ]:
df = spark.read.load('/data/products.csv', format='csv', header=True)
display(df.limit(10))

## Load the file data into a delta table

You can persist the data in the dataframe in Delta format using the following code:

In [ ]:
delta_table_path = "/delta/products-delta"
df.write.format("delta").save(delta_table_path)

The data for a delta lake table is stored in Parquet format. A log file is also created to track modifications made to the data.

Use the following shell commands to view the contents of the folder where the delta data has been saved.

In [ ]:
%sh
ls /dbfs/delta/products-delta

The file data in Delta format can be loaded into a **DeltaTable** object, which you can use to view and update the data in the table. Run the following cell to update the data; reducing the price of product 771 by 10%.

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Create a deltaTable object
deltaTable = DeltaTable.forPath(spark, delta_table_path)

# Update the table (reduce price of product 771 by 10%)
deltaTable.update(
    condition = "ProductID == 771",
    set = { "ListPrice": "ListPrice * 0.9" })

# View the updated data as a dataframe
deltaTable.toDF().show(10)

The update is persisted to the data in the delta folder, and will be reflected in any new dataframe loaded from that location:

In [ ]:
new_df = spark.read.format("delta").load(delta_table_path)
new_df.show(10)

Data modifications are logged, enabling you to use the *time-travel* capabilities of Delta Lake to view previous versions of the data. For example, use the following code to view the original version of the product data:

In [ ]:
new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
new_df.show(10)

The log contains a full history of modifications to the data. Use the following code to see a record of the last 10 changes:

In [ ]:
deltaTable.history(10).show(10, False, True)

## Create catalog tables

So far you've worked with delta tables by loading data from the folder containing the parquet files on which the table is based. You can define *catalog tables* that encapsulate the data and provide a named table entity that you can reference in SQL code. Spark supports two kinds of catalog tables for delta lake:

- *External* tables that are defined by the path to the parquet files containing the table data.
- *Managed* tables, that are defined in the Hive metastore for the Spark cluster

### Create an external table

The following code creates a new database named **AdventureWorks** and then creates an external tabled named **ProductsExternal** in that database based on the path to the Delta files you defined previously.

In [ ]:
spark.sql("CREATE DATABASE AdventureWorks")
spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

Note that the **Location** property of the new table is the path you specified.

You can query the new table by using a SQL `SELECT` statement, like this:

In [ ]:
%sql
USE AdventureWorks;

SELECT * FROM ProductsExternal;

### Create a managed table

A *managed* table stores its data files in the Hive metastore for the Spark cluster.

Run the following code to create (and then describe) a managed tabled named **ProductsManaged** based on the dataframe you originally loaded from the **products.csv** file (before you updated the price of product 771).

In [ ]:
df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

You did not specify a path for the parquet files used by the table - this is managed for you in the Hive metastore, and shown in the **Location** property in the table description (in the **dbfs:/user/hive/warehouse/** path).

From the SQL user's perspective, there's no difference between external and managed tables when it comes to querying them with a `SELECT` statement:

In [ ]:
%sql
USE AdventureWorks;

SELECT * FROM ProductsManaged;

### Compare external and managed tables

let's explore the differences between external and managed tables.

First, use the following code to list the tables in the **AdventureWorks** database:

In [ ]:
%sql
USE AdventureWorks;

SHOW TABLES;

Now let's take a look at the folders on which these tables are based:

In [ ]:
%sh
echo "External table:"
ls /dbfs/delta/products-delta
echo
echo "Managed table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

What happens if we use a `DROP` statement to delete these tables from the database?

In [ ]:
%sql

USE AdventureWorks;

DROP TABLE IF EXISTS ProductsExternal;
DROP TABLE IF EXISTS ProductsManaged;

SHOW TABLES;

The metadata for both tables has been removed from the database; but what about the delta files?

In [ ]:
%sh
echo "External table:"
ls /dbfs/delta/products-delta
echo
echo "Managed table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

The files for the managed table are deleted automatically when the table is dropped. However, the files for the external table remain. Dropping an external table only removes the table metadata from the database; it does not delete the data files.

You can use the following code to create a new table in the database that is based on the delta files in the **products-delta** folder:

In [ ]:
%sql
USE AdventureWorks;

CREATE TABLE Products
USING DELTA
LOCATION '/delta/products-delta';

Now you can query the new table

In [ ]:
%sql
USE AdventureWorks;

SELECT * FROM Products;